In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
data = pd.read_csv('/kaggle/input/loan-data/loan_data.csv')
data.head()

In [ ]:
data.info()

In [ ]:
#Check for missing values
data.isnull().sum()

In [ ]:
cat_feats = ['purpose']
X = pd.get_dummies(data,columns=cat_feats,drop_first=True)
X.head()

In [ ]:
#Exploratory Data Analysis
plt.figure(figsize = (20, 10))
sns.heatmap(X.corr(), annot= True)

In [ ]:
#Target: credit_policy = 1 if meets underwriting standars, 0 if not
#Fico score stands out as a high correlation with credit_policy 

sns.kdeplot(data=X, x='fico',hue="credit.policy", fill=True)

In [ ]:
#Likewise, inquiries have a negative correlation with credit.policy
sns.kdeplot(data=X, x='inq.last.6mths',hue="credit.policy", fill=True)

In [ ]:
sns.kdeplot(data=X, x='revol.util',hue="credit.policy", fill=True)

In [ ]:
#Majority of dataset has approved underwriting
plt.pie(x=X['credit.policy'].value_counts(), labels = X['credit.policy'].value_counts().index,radius=2, autopct="%1.0f%%")

In [ ]:
sns.kdeplot(data=X, x='days.with.cr.line',hue="credit.policy", fill=True)

In [ ]:
sns.kdeplot(data=X, x='int.rate',hue="credit.policy", fill=True)

In [ ]:
sns.kdeplot(data=X, x='installment',hue="credit.policy", fill=True)


In [ ]:
import plotly.express as px
fig = px.scatter(X, x="fico", y = "int.rate", color = 'credit.policy')
fig.show()
#Negative relationship between interest rate and FICO

In [ ]:
#Begin predictive modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix



In [ ]:
#Get data
y = X['credit.policy']
X = X.drop(['credit.policy'],axis =1)

print(f'X : {X.shape}')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .4, random_state = 101)

In [ ]:
#Build Random Forest model on train dataset
pipe_RandFor = Pipeline([('RandomForest',RandomForestClassifier())])

rf_model = pipe_RandFor.fit(X_train, y_train)

In [ ]:
#Evaluate model accuracy
accuracy = round(rf_model.score(X_test,y_test) * 100,2)
print("Accuracy: {}".format(accuracy))

In [ ]:
predictions = rf_model.predict(X_test)

print(classification_report(y_test,predictions))


print(confusion_matrix(y_test,predictions))




In [ ]:
#Build Logistic Regression Model
pipe_LogReg = Pipeline([('scaler',MinMaxScaler()),
                        ('LogisticRegression',LogisticRegression(C=2))])

In [ ]:
lr_model = pipe_LogReg.fit(X_train,y_train)

In [ ]:
prediction = lr_model.predict(X_test)
score = round(accuracy_score(y_test,prediction),2)*100
print("Accuracy: {}".format(score))